In [1]:
trait RNG {
    def nextInt: (Int, RNG)
}

case class SimpleRNG(seed: Long) extends RNG { 
    def nextInt: (Int, RNG) = {
        val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL 
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)
    }
}


defined trait RNG
defined class SimpleRNG


In [2]:
var nowRng = (0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 10) {
    nowRng = nowRng._2.nextInt
    var nextRng = nowRng._2
    println(nowRng._1)
}

384748
-1151252339
-549383847
1612966641
-883454042
1563994289
1331515492
-234691648
672332705
-2039128390


null

In [3]:
// id : ex_6.1

/* Write a function that uses RNG.nextInt to generate a 
   random integer between 0 and Int.maxValue (inclusive). 
   Make sure to handle the corner case when nextInt returns 
   Int.MinValue, which doesn’t have a non-negative counterpart.
*/

// simple idea, map every negative integer to it's postive.
// watching the edge case as indicated
def nextNaturalInt(rng : RNG) : (Int, RNG) = {
    val nextRng = rng.nextInt
    if (nextRng._1 == Int.MinValue) {
        (0, nextRng._2)
    } else {
        (Math.abs(nextRng._1), nextRng._2)
    }
}

nextNaturalInt: (rng: RNG)(Int, RNG)


In [4]:
// id : try ex_6.1

case class RngValue(current : Int) extends RNG {
    def nextInt : (Int, RNG) = {
        (current, this)
    }
}

println(nextNaturalInt(RngValue(Int.MinValue)))
println(nextNaturalInt(RngValue(Int.MinValue+1)))
println(nextNaturalInt(RngValue(-1)))
println(nextNaturalInt(RngValue(0)))
println(nextNaturalInt(RngValue(1)))
println(nextNaturalInt(RngValue(Int.MaxValue-1)))
println(nextNaturalInt(RngValue(Int.MaxValue)))

var nowRng = (0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = nextNaturalInt(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}



(0,RngValue(-2147483648))
(2147483647,RngValue(-2147483647))
(1,RngValue(-1))
(0,RngValue(0))
(1,RngValue(1))
(2147483646,RngValue(2147483646))
(2147483647,RngValue(2147483647))
384748
1151252339
549383847
1612966641
883454042
1563994289
1331515492
234691648
672332705
2039128390
1888584533
294927845
1517050556
92416162
1713389258


null

In [5]:
// id : ex_6.2

def double(rng : RNG) : (Double, RNG) = {
    val nextRngInt = nextNaturalInt(rng)
    (nextRngInt._1.toDouble / (Int.MaxValue.toDouble + 1), nextRngInt._2)
}

double: (rng: RNG)(Double, RNG)


In [6]:
// id : try ex_6.2

println(double(RngValue(0)))
println(double(RngValue(Int.MaxValue)))

var nowRng = (0.0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = double(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}

(0.0,RngValue(0))
(0.9999999995343387,RngValue(2147483647))
1.7916224896907806E-4
0.5360936461947858
0.25582678942009807
0.7510961224325001
0.41139034647494555
0.7282915939576924
0.6200352180749178
0.10928681492805481
0.3130793129093945
0.9495431510731578
0.8794407048262656
0.1373364799655974
0.7064317148178816
0.043034628964960575
0.7978590475395322


null

In [7]:
// id : ex_6.3

def intDouble(rng : RNG) : ((Int, Double), RNG) = {
    val (int, rng1) = nextNaturalInt(rng)
    val (dbl, rng2) = double(rng1)
    ((int, dbl), rng2)
}

def doubleInt(rng : RNG) : ((Double, Int), RNG) = {
    val ((int, dbl), rng1) = intDouble(rng)
    ((dbl, int), rng1)
}

def double3(rng : RNG) : ((Double, Double, Double), RNG) = {
    val (dbl1, rng1) = double(rng)
    val (dbl2, rng2) = double(rng1)
    val (dbl3, rng3) = double(rng2)
    ((dbl1, dbl2, dbl3), rng3)
}

intDouble: (rng: RNG)((Int, Double), RNG)
doubleInt: (rng: RNG)((Double, Int), RNG)
double3: (rng: RNG)((Double, Double, Double), RNG)


In [8]:
// id : try ex_6.3
val rng20 = RngValue(20)
println(intDouble(rng20))
println(doubleInt(rng20))
println(double3(rng20))

((20,9.313225746154785E-9),RngValue(20))
((9.313225746154785E-9,20),RngValue(20))
((9.313225746154785E-9,9.313225746154785E-9,9.313225746154785E-9),RngValue(20))


null

In [9]:
// id : ex_6.4

def ints(count: Int)(rng : RNG) : (List[Int], RNG) = {
    if (count <= 0) {
        (List(), rng)
    } else {
        val (i, rng1) = nextNaturalInt(rng)
        val rem = ints(count-1)(rng1)
        (i::rem._1, rem._2)
    }
}

ints: (count: Int)(rng: RNG)(List[Int], RNG)


In [10]:
// id : try ex_6.4

ints(5)(SimpleRNG(1))

(List(384748, 1151252339, 549383847, 1612966641, 883454042),SimpleRNG(223576932655868))

In [11]:
type Rand[+A] = RNG => (A, RNG)

def map[A,B](s: Rand[A])(f: A => B): Rand[B] = rng => {
    val (a, rng2) = s(rng)
    (f(a), rng2)
}

defined type alias Rand
map: [A, B](s: Rand[A])(f: A => B)Rand[B]


In [12]:
// id : ex_6.5

def double(rng : RNG) : (Double, RNG) =
    map(rng => {
        val (i, r) = nextNaturalInt(rng)
        (i.toDouble, r)
    })(i => i.toDouble / (Int.MaxValue.toDouble + 1))(rng)

double: (rng: RNG)(Double, RNG)


In [13]:
// id : try ex_6.5

println(double(RngValue(0)))
println(double(RngValue(Int.MaxValue)))

var nowRng = (0.0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = double(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}

(0.0,RngValue(0))
(0.9999999995343387,RngValue(2147483647))
1.7916224896907806E-4
0.5360936461947858
0.25582678942009807
0.7510961224325001
0.41139034647494555
0.7282915939576924
0.6200352180749178
0.10928681492805481
0.3130793129093945
0.9495431510731578
0.8794407048262656
0.1373364799655974
0.7064317148178816
0.043034628964960575
0.7978590475395322


null

In [14]:
// id : ex_6.6

def map2[A,B,C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C): Rand[C] = rng => {
    val (a, rng1) = ra(rng)
    val (b, rng2) = rb(rng1)
    (f(a,b), rng2)
}

map2: [A, B, C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C)Rand[C]


In [15]:
// id : try ex_6.6

def both[A,B](ra: Rand[A], rb: Rand[B]): Rand[(A,B)] = map2(ra, rb)((_, _))

val randIntDouble: Rand[(Int, Double)] = both(nextNaturalInt, double)
val randDoubleInt: Rand[(Double, Int)] = both(double, nextNaturalInt)

println(randIntDouble(SimpleRNG(1)))
println(randDoubleInt(SimpleRNG(1)))

((384748,0.5360936461947858),SimpleRNG(206026503483683))
((1.7916224896907806E-4,1151252339),SimpleRNG(206026503483683))


null

In [16]:
// id : ex_6.7

// Rand[A] = rng : RNG => (A, rng)
def sequence[A](fs: List[Rand[A]]) : Rand[List[A]] = {
    rng => {
        fs match {
            case Nil => (List().asInstanceOf[List[A]], rng)
            case r2s :: rest => {
                val (r2sa, r2sr) = r2s(rng)
                val rem = sequence(rest)(r2sr)
                (r2sa :: rem._1, rem._2)
            }
        }
    }
}

def ints(count: Int)(rng : RNG) : (List[Int], RNG) = sequence(List.fill(count)(nextNaturalInt _))(rng)

sequence: [A](fs: List[Rand[A]])Rand[List[A]]
ints: (count: Int)(rng: RNG)(List[Int], RNG)


In [17]:
// id : try ex_6.7

ints(5)(SimpleRNG(1))

(List(384748, 1151252339, 549383847, 1612966641, 883454042),SimpleRNG(223576932655868))

In [18]:
// id : ex_6.8

def flatMap[A,B](f: Rand[A])(g: A => Rand[B]): Rand[B] = {
    rng => {
        val (a, arng) = f(rng)
        g(a)(arng)
    }
}

def nonNegativeLessThan(n: Int) : Rand[Int] = 
    flatMap(nextNaturalInt)(i => {
        val mod = i % n
        if (i + (n-1) - mod >= 0)
        (r: RNG) => (mod, r)
        else (r: RNG) => nonNegativeLessThan(n)(r)
    })


flatMap: [A, B](f: Rand[A])(g: A => Rand[B])Rand[B]
nonNegativeLessThan: (n: Int)Rand[Int]


In [19]:
// id : try ex_6.8

nonNegativeLessThan(1000)(SimpleRNG(1))

(748,SimpleRNG(25214903928))

In [20]:
// id : ex_6.9.1

def map[A,B](s: Rand[A])(f: A => B) : Rand[B] = 
    flatMap(s)(a => rng => (f(a), rng))

map: [A, B](s: Rand[A])(f: A => B)Rand[B]


In [21]:
// id : try ex_6.9.1

def double(rng : RNG) : (Double, RNG) =
    map(rng => {
        val (i, r) = nextNaturalInt(rng)
        (i.toDouble, r)
    })(i => i.toDouble / (Int.MaxValue.toDouble + 1))(rng)

println(double(RngValue(0)))
println(double(RngValue(Int.MaxValue)))

var nowRng = (0.0, SimpleRNG(1).asInstanceOf[RNG])
for(i <- 1 to 15) {
    nowRng = double(nowRng._2)
    var nextRng = nowRng._2
    println(nowRng._1)
}

(0.0,RngValue(0))
(0.9999999995343387,RngValue(2147483647))
1.7916224896907806E-4
0.5360936461947858
0.25582678942009807
0.7510961224325001
0.41139034647494555
0.7282915939576924
0.6200352180749178
0.10928681492805481
0.3130793129093945
0.9495431510731578
0.8794407048262656
0.1373364799655974
0.7064317148178816
0.043034628964960575
0.7978590475395322


null

In [22]:
// id : ex_6.9.2

def map2[A,B,C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C): Rand[C] = 
    flatMap(ra)(a => flatMap(rb)(b => rng2 => (f(a,b), rng2)))

map2: [A, B, C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C)Rand[C]


In [23]:
// id : try ex_6.9.2

def both[A,B](ra: Rand[A], rb: Rand[B]): Rand[(A,B)] = map2(ra, rb)((_, _))

val randIntDouble: Rand[(Int, Double)] = both(nextNaturalInt, double)
val randDoubleInt: Rand[(Double, Int)] = both(double, nextNaturalInt)

println(randIntDouble(SimpleRNG(1)))
println(randDoubleInt(SimpleRNG(1)))

((384748,0.5360936461947858),SimpleRNG(206026503483683))
((1.7916224896907806E-4,1151252339),SimpleRNG(206026503483683))


null

In [24]:
// id : ex_6.10

object State {
    def unit[S,A](a: A):  State[S,A] = State(s => (a, s))
    
    def sequence[S, A](fs: List[State[S,A]]) : State[S,List[A]] =
        State((s : S) => {
            fs match {
                case Nil => (List().asInstanceOf[List[A]], s)
                case s2s :: rest => {
                    val (s2sa, s2ss) = s2s.run(s)
                    val rem = sequence(rest).run(s2ss)
                    (s2sa :: rem._1, rem._2)
                }
            }
        })
    
    // From the section just above exercise 6.11
    def get[S]: State[S,S] = State(s => (s,s))
    
    def set[S](s: S): State[S, Unit] = State(_ => ((), s))
    
    def modify[S](f : S => S): State[S, Unit] = for {
        s <- get
        _ <- set(f(s))
    } yield ()
}

case class State[S,A](run: S => (A,S)) {
    
    def flatMap[B](g: A => State[S, B]): State[S, B] = {
        State((s : S) => {
            val (a, as) = run(s)
            g(a).run(as)
        })
    }
    
    def map[B](f: A => B) : State[S,B] =
        flatMap(a => State.unit(f(a)))
    
    def map2[B,C](sb: State[S,B])(f: (A, B) => C): State[S,C] =
        flatMap(a => sb.flatMap(b => State.unit(f(a,b))))
    
}




defined object State
defined class State


In [25]:
// id : try ex_6.10.1
val rng = SimpleRNG(1)
State(double).map(_.toString).run(rng)

(1.7916224896907806E-4,SimpleRNG(25214903928))

In [26]:
// id : try ex_6.10.2
type Rand[A] = State[RNG, A]

def nextNaturalInt : Rand[Int] = 
    State( rng => {
        val nextRng = rng.nextInt
        if (nextRng._1 == Int.MinValue) {
            (0, nextRng._2)
        } else {
            (Math.abs(nextRng._1), nextRng._2)
        }    
    })

def ints(count: Int) : Rand[List[Int]] = State.sequence(List.fill(count)(nextNaturalInt))

println(ints(4).run(SimpleRNG(1)))

(List(384748, 1151252339, 549383847, 1612966641),SimpleRNG(105707381795861))


null

In [27]:
// id : try ex_6.10.3
def nonNegativeLessThan(n: Int) : Rand[Int] = 
    nextNaturalInt.flatMap(i => State(s => {
        val mod = i % n
        if (i + (n-1) - mod >= 0) (mod, s)
        else nonNegativeLessThan(n).run(s)
    }))

println(nonNegativeLessThan(10).run(SimpleRNG(1)))

(8,SimpleRNG(25214903928))


null

In [28]:
// id : try ex_6.10.4
def ns(n: Int) : Rand[List[Int]] = 
    nonNegativeLessThan(n).flatMap(x =>
    nonNegativeLessThan(n).flatMap(y =>
    ints(x).map(xs =>
    xs.map(_ % y))))

ns: (n: Int)Rand[List[Int]]


In [29]:
// id : try ex_6.10.5
for(i <- 1 to 10) {
    println("Seed " + i, ns(20).run(SimpleRNG(i)))
}

(Seed 1,(List(6, 2, 3, 17, 14, 0, 16, 16),SimpleRNG(147838658590923)))
(Seed 2,(List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),SimpleRNG(73378791630839)))
(Seed 3,(List(0, 5, 0, 1, 5, 5),SimpleRNG(242528784466859)))
(Seed 4,(List(7, 0, 7, 6, 1, 3, 2, 9, 9, 2, 0, 4, 8, 3, 8),SimpleRNG(139330556974351)))
(Seed 5,(List(2, 4, 5, 2),SimpleRNG(128185544502587)))
(Seed 6,(List(8, 3, 3, 2, 9, 8, 11, 2, 12, 7, 7, 2, 5),SimpleRNG(232484593450071)))
(Seed 7,(List(1, 3),SimpleRNG(160153793685115)))
(Seed 8,(List(14, 11, 16, 9, 2, 7, 7, 15, 12, 7, 7),SimpleRNG(85457628633295)))
(Seed 9,(List(),SimpleRNG(163169759544427)))
(Seed 10,(List(0, 4, 2, 4, 4, 1, 4, 4, 4),SimpleRNG(141917030058359)))


null

In [30]:
// id : try ex_6.10.6
def nsFor(n: Int) : Rand[List[Int]] = 
    for { 
        x <- nonNegativeLessThan(n)
        y <- nonNegativeLessThan(n)
        xs <- ints(x)
    } yield xs.map(_ % y)

nsFor: (n: Int)Rand[List[Int]]


In [31]:
// id : try ex_6.10.7
for(i <- 1 to 10) {
    println("Seed " + i, nsFor(20).run(SimpleRNG(i)))
}

(Seed 1,(List(6, 2, 3, 17, 14, 0, 16, 16),SimpleRNG(147838658590923)))
(Seed 2,(List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),SimpleRNG(73378791630839)))
(Seed 3,(List(0, 5, 0, 1, 5, 5),SimpleRNG(242528784466859)))
(Seed 4,(List(7, 0, 7, 6, 1, 3, 2, 9, 9, 2, 0, 4, 8, 3, 8),SimpleRNG(139330556974351)))
(Seed 5,(List(2, 4, 5, 2),SimpleRNG(128185544502587)))
(Seed 6,(List(8, 3, 3, 2, 9, 8, 11, 2, 12, 7, 7, 2, 5),SimpleRNG(232484593450071)))
(Seed 7,(List(1, 3),SimpleRNG(160153793685115)))
(Seed 8,(List(14, 11, 16, 9, 2, 7, 7, 15, 12, 7, 7),SimpleRNG(85457628633295)))
(Seed 9,(List(),SimpleRNG(163169759544427)))
(Seed 10,(List(0, 4, 2, 4, 4, 1, 4, 4, 4),SimpleRNG(141917030058359)))


null

In [32]:
// id : ex_6.11.1

sealed trait Input
case object Coin extends Input
case object Turn extends Input

var DEBUG_UGLY = false

case class Machine(locked: Boolean, candies: Int, coins: Int) {
    
    val content = ((candies, coins), this)

    // - A machine that’s out of candy ignores all inputs.
    def takeInput(input : Input) : ((Int, Int), Machine) = {
        val after = if (candies <= 0 ) content
            else input match {
                case Coin => coin()
                case Turn => turn()
            }
        
        val u = if(DEBUG_UGLY) println((input, this, "=>", after).productIterator.mkString("\t"))
            else ()
        after
    }
    
    /*
    - Inserting a coin into a locked machine will cause it to unlock 
      if there is any candy left.
    - Inserting a coin into an unlocked machine does "nothing":
      (Here, we assume we reject the coin input and it does not add
       to the total number of coins).
    */
    private def coin() : ((Int, Int), Machine) = {
        if (locked) {
            ((candies, coins+1), Machine(false, candies, coins+1))
        } else {
            content
        }
    }
    
    /*
    - Turning the knob on an unlocked machine will cause it to dispense 
      candy and become locked.
    - Turning the knob on a locked machine does nothing
    */
    private def turn() : ((Int, Int), Machine) = {
        if (!locked) {
            ((candies-1, coins), Machine(true, candies-1, coins))
        } else {
            content
        }
    }
}



defined trait Input
defined object Coin
defined object Turn
DEBUG_UGLY: Boolean = false
defined class Machine


In [33]:
// id : ex_6.11.2

/*
The method simulateMachine should operate the machine based on the 
list of inputs and return the number of coins and candies left in 
the machine at the end. For example, if the input Machine has 10 
coins and 5 candies, and a total of 4 candies are successfully 
bought, the output should be (14, 1).
*/

def simulationMachine(inputs : List[Input], initState : Machine) : (Int, Int) = {
    val states = inputs.map(i => State((s: Machine) => s.takeInput(i)))
    val state : State[Machine, (Int, Int)] = for {
        _ <- State.sequence(states)
        m <- State.get
    } yield (m.candies, m.coins)
    state.run(initState)._1
}

def successTries(count: Int) : List[Input] = 
    List.fill(count)(List(Coin, Turn)).flatten

simulationMachine(successTries(4), Machine(true, 5, 10))

(1,14)

In [34]:
// id : try ex_6.11.1
/*
def ns(n: Int) : Rand[List[Int]] = 
    nonNegativeLessThan(n).flatMap(x =>
    nonNegativeLessThan(n).flatMap(y =>
    ints(x).map(xs =>
    xs.map(_ % y))))

              becomes
---------vvvvvvvvvvvvvvvv----------

def nsFor(n: Int) : Rand[List[Int]] = 
    for { 
        x <- nonNegativeLessThan(n)
        y <- nonNegativeLessThan(n)
        xs <- ints(x)
    } yield xs.map(_ % y)
*/

/* De-sugar the following:
val initState = Machine(true, 5, 10)
val inputs = successTries(4)
val states = inputs.map(i => State((s: Machine) => s.takeInput(i)))
val state : State[Machine, (Int, Int)] = for {
    _ <- State.sequence(states)
    m <- State.get
} yield (m.candies, m.coins)
state.run(initState)
*/

val initState = Machine(true, 5, 10)
val inputs = successTries(4)
val states = inputs.map(i => State((s: Machine) => s.takeInput(i)))
val state : (Int, Int) = 
    State.sequence(states).flatMap(_ => State.get).map(m => (m.candies, m.coins)).run(initState)._1


(1,14)

In [35]:
// id : try ex_6.11.2
DEBUG_UGLY = true
// Sequence actually runs our machine, tieing the output of the last state to the next
State.sequence(states).run(initState)

Coin	Machine(true,5,10)	=>	((5,11),Machine(false,5,11))
Turn	Machine(false,5,11)	=>	((4,11),Machine(true,4,11))
Coin	Machine(true,4,11)	=>	((4,12),Machine(false,4,12))
Turn	Machine(false,4,12)	=>	((3,12),Machine(true,3,12))
Coin	Machine(true,3,12)	=>	((3,13),Machine(false,3,13))
Turn	Machine(false,3,13)	=>	((2,13),Machine(true,2,13))
Coin	Machine(true,2,13)	=>	((2,14),Machine(false,2,14))
Turn	Machine(false,2,14)	=>	((1,14),Machine(true,1,14))


(List((5,11), (4,11), (4,12), (3,12), (3,13), (2,13), (2,14), (1,14)),Machine(true,1,14))

In [36]:
// id : try ex_6.11.3
// The last return value being the list of our coins / candy is nice, but
// we'd have to iterate over that list to the end in order to get our final state.
// That information is already encoded in Machine() which is passed along in the
// tuple pair being carried forward in the plumbing of flatMap.  
// We can just call State.get to make the mapped value returned out be the (S, S)
// pair, or in other words the final Machine() value.
State.sequence(states).flatMap(_ => State.get).run(initState)

Coin	Machine(true,5,10)	=>	((5,11),Machine(false,5,11))
Turn	Machine(false,5,11)	=>	((4,11),Machine(true,4,11))
Coin	Machine(true,4,11)	=>	((4,12),Machine(false,4,12))
Turn	Machine(false,4,12)	=>	((3,12),Machine(true,3,12))
Coin	Machine(true,3,12)	=>	((3,13),Machine(false,3,13))
Turn	Machine(false,3,13)	=>	((2,13),Machine(true,2,13))
Coin	Machine(true,2,13)	=>	((2,14),Machine(false,2,14))
Turn	Machine(false,2,14)	=>	((1,14),Machine(true,1,14))


(Machine(true,1,14),Machine(true,1,14))

In [37]:
// id : try ex_6.11.4
// For those curious and for confirmation this is true, we see the prior
// value output of sequence in the captured value t, and what ends is our
// (state, state) pair after the run
State.sequence(states).flatMap(t => {
    println("The value of t: " + t)
    State.get
}).run(initState)

Coin	Machine(true,5,10)	=>	((5,11),Machine(false,5,11))
Turn	Machine(false,5,11)	=>	((4,11),Machine(true,4,11))
Coin	Machine(true,4,11)	=>	((4,12),Machine(false,4,12))
Turn	Machine(false,4,12)	=>	((3,12),Machine(true,3,12))
Coin	Machine(true,3,12)	=>	((3,13),Machine(false,3,13))
Turn	Machine(false,3,13)	=>	((2,13),Machine(true,2,13))
Coin	Machine(true,2,13)	=>	((2,14),Machine(false,2,14))
Turn	Machine(false,2,14)	=>	((1,14),Machine(true,1,14))
The value of t: List((5,11), (4,11), (4,12), (3,12), (3,13), (2,13), (2,14), (1,14))


(Machine(true,1,14),Machine(true,1,14))

In [38]:
// id : try ex_6.11.5
// Finally, the full string passes that state value forward via map
// The run will always return back the (Value, State) pair, so if you
// want to throw away the final state you'd can use the _1 on the result.
State.sequence(states).flatMap(_ => State.get).map(m => m.content._1).run(initState)._1

Coin	Machine(true,5,10)	=>	((5,11),Machine(false,5,11))
Turn	Machine(false,5,11)	=>	((4,11),Machine(true,4,11))
Coin	Machine(true,4,11)	=>	((4,12),Machine(false,4,12))
Turn	Machine(false,4,12)	=>	((3,12),Machine(true,3,12))
Coin	Machine(true,3,12)	=>	((3,13),Machine(false,3,13))
Turn	Machine(false,3,13)	=>	((2,13),Machine(true,2,13))
Coin	Machine(true,2,13)	=>	((2,14),Machine(false,2,14))
Turn	Machine(false,2,14)	=>	((1,14),Machine(true,1,14))


(1,14)